In [1]:

from elftools.elf.elffile import ELFFile
from elftools.dwarf.descriptions import (
    describe_DWARF_expr, set_global_machine_arch)
from elftools.dwarf.locationlists import (
    LocationEntry, LocationExpr, LocationParser)



def show_loclist(loclist, dwarfinfo, indent, cu_offset):
    """ Display a location list nicely, decoding the DWARF expressions
        contained within.
    """
    d = []
    for loc_entity in loclist:
        if isinstance(loc_entity, LocationEntry):
            d.append('%s <<%s>>' % (
                loc_entity,
                describe_DWARF_expr(loc_entity.loc_expr, dwarfinfo.structs, cu_offset)))
        else:
            d.append(str(loc_entity))
    return '\n'.join(indent + s for s in d)



In [2]:
def get_DIE_at_offset(CU, offset):
        for die in CU.iter_DIEs():
            if die.offset == CU.cu_offset+offset:
                return die 
        return None

def get_type_name(CU, offset):#get_DIE_at_offset(CU,attr.value)
    die = get_DIE_at_offset(CU, offset)
    
    if die.tag == 'DW_TAG_pointer_type':
        for _attr in die.attributes.values():
            if _attr.name== "DW_AT_type":
                return "*"+get_type_name(CU,_attr.value) 
        
    
    for attr in die.attributes.values():
        if attr.name== "DW_AT_name":
            return attr.value.decode("utf-8")
    
    


In [3]:
from collections import defaultdict
def line_entry_mapping(line_program):
    filename_map = defaultdict(int)

    # The line program, when decoded, returns a list of line program
    # entries. Each entry contains a state, which we'll use to build
    # a reverse mapping of filename -> #entries.
    lp_entries = line_program.get_entries()
    for lpe in lp_entries:
        # We skip LPEs that don't have an associated file.
        # This can happen if instructions in the compiled binary
        # don't correspond directly to any original source file.
        if not lpe.state or lpe.state.file == 0:
            continue
        filename = lpe_filename(line_program, lpe.state.file)
        filename_map[filename] += 1

    # for filename, lpe_count in filename_map.items():
    #     print("    filename=%s -> %d entries" % (filename, lpe_count))
    return filename_map

def lpe_filename(line_program, file_index):

    lp_header = line_program.header
    file_entries = lp_header["file_entry"]
    

    # File and directory indices are 1-indexed.
    file_entry = file_entries[file_index - 1]
    dir_index = file_entry["dir_index"]

    # A dir_index of 0 indicates that no absolute directory was recorded during
    # compilation; return just the basename.
    if dir_index == 0:
        return file_entry.name.decode()

    directory = lp_header["include_directory"][dir_index - 1]
    return posixpath.join(directory, file_entry.name).decode()

In [4]:

FUNC_PARAMS = {}

def process_file(filename):
    print('Processing file:', filename)
    with open(filename, 'rb') as f:
        elffile = ELFFile(f)

        if not elffile.has_dwarf_info():
            print('  file has no DWARF info')
            return

        # get_dwarf_info returns a DWARFInfo context object, which is the
        # starting point for all DWARF-based processing in pyelftools.
        dwarfinfo = elffile.get_dwarf_info()

        # The location lists are extracted by DWARFInfo from the .debug_loc
        # section, and returned here as a LocationLists object.
        location_lists = dwarfinfo.location_lists()

        # This is required for the descriptions module to correctly decode
        # register names contained in DWARF expressions.
        set_global_machine_arch(elffile.get_machine_arch())

        # Create a LocationParser object that parses the DIE attributes and
        # creates objects representing the actual location information.
        loc_parser = LocationParser(location_lists)
        
        section_offset = dwarfinfo.debug_info_sec.global_offset
        # Offset of the .debug_info section in the stream
        

        
        for CU in dwarfinfo.iter_CUs():
            
            #get source file name
            line_program = dwarfinfo.line_program_for_CU(CU)
            filename_map = line_entry_mapping(line_program)
            CU_src_file =  list(filename_map.items())[0][0]
            
            CU_dictionary_key = CU_src_file 
            
            FUNC_PARAMS[CU_dictionary_key] = {}
            
            print('  Found a compile unit at offset %s, length %s' % (
                CU.cu_offset, CU['unit_length']))

            # A CU provides a simple API to iterate over all the DIEs in it.
            die_depth = 0
            are_DIEs_of_function = False
            FUNC_name = None
            for DIE in CU.iter_DIEs():
                
                ############################################################
                #############   Prasing Function DIEs start ################
                
                if DIE.tag == 'DW_TAG_subprogram':
                    are_DIEs_of_function = True

                    for attr in DIE.attributes.values():
                        if attr.name == "DW_AT_name": #FUNC NAME
                            FUNC_name = attr.value.decode("utf-8")
                            FUNC_PARAMS[CU_dictionary_key][FUNC_name] ={}

                            
                if DIE.tag == 'DW_TAG_formal_parameter':
                    tags = [attr.name for attr in DIE.attributes.values()]
                    PARAM_name = None
                    if "DW_AT_name" in tags:
                        for attr in DIE.attributes.values():
                            if  attr.name == "DW_AT_location":
                                print(attr.value)
                            if attr.name == "DW_AT_name": # VAR NAME 
                                PARAM_name = attr.value.decode("utf-8")
                                FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name] = {}
                                print(PARAM_name)
                            if attr.name == "DW_AT_type":
                                FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name] = {'type':get_type_name(CU,attr.value) , 'kind':'parameter'}
                        print("__________________\n")     
                                
                if DIE.tag == 'DW_TAG_variable':
                    PARAM_name = None
                    for attr in DIE.attributes.values():
                        if  attr.name == "DW_AT_location":
                            print(attr.value)
                        if attr.name == "DW_AT_name": # VAR NAME 
                            PARAM_name = attr.value.decode("utf-8")
                            print(PARAM_name)
                            FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name] = {}

                        if attr.name == "DW_AT_type":
                            FUNC_PARAMS[CU_dictionary_key][FUNC_name][PARAM_name] = {'type':get_type_name(CU,attr.value) , 'kind':'variable'}

                    print("__________________\n")

                ###############################################
                #############  parsing  Function DIEs ends ################
                


                
                if DIE.is_null(): #https://chromium.googlesource.com/chromiumos/third_party/pyelftools/+/25a77f7738d7fe824f2ed4d33a123136b9d8e88a/scripts/readelf.py
                    are_DIEs_of_function = False
                    FUNC_name = None
                    
                    die_depth -= 1
                    continue
                if DIE.has_children:
                    die_depth += 1
                    

filename = './../../binaries/c_many/stacktest'
process_file(filename)

Processing file: ./../../binaries/c_many/stacktest
  Found a compile unit at offset 0, length 890
a
[145, 92]
__________________

b
[145, 88]
__________________

d
[145, 96]
__________________

e
[145, 100]
__________________

f
[145, 104]
__________________

g
[145, 108]
__________________

stk_ptr
[145, 72]
__________________

int1
[145, 84]
__________________

int2
[145, 88]
__________________

number
[145, 92]
__________________

stk
[145, 96]
__________________

tmp
[145, 104]
__________________

a_int
[145, 236, 126]
__________________

b_float
[145, 232, 126]
__________________

c_double
[145, 224, 126]
__________________

char_pointer
[145, 216, 126]
__________________

e_int
[145, 252, 126]
__________________

f_float
[145, 128, 127]
__________________

g_double
[145, 136, 127]
__________________

x1
__________________

x2
__________________

x3
__________________

x4
__________________

x5
__________________

x6
[145, 132, 127]
__________________

int_arr
[145, 144, 127]
____

In [5]:
FUNC_PARAMS 


{'stack.c': {'free': {},
  'printf': {},
  'malloc': {},
  'fake': {'a': {'type': 'int', 'kind': 'parameter'},
   'b': {'type': 'int', 'kind': 'parameter'},
   'd': {'type': 'int', 'kind': 'variable'},
   'e': {'type': 'int', 'kind': 'variable'},
   'f': {'type': 'int', 'kind': 'variable'},
   'g': {'type': 'int', 'kind': 'variable'}},
  'pop': {'stk_ptr': {'type': '**stack', 'kind': 'parameter'},
   'int1': {'type': 'int', 'kind': 'variable'},
   'int2': {'type': 'int', 'kind': 'variable'},
   'number': {'type': 'int', 'kind': 'variable'},
   'stk': {'type': '*stack', 'kind': 'variable'},
   'tmp': {'type': '*stack', 'kind': 'variable'}},
  'dummy': {'a_int': {'type': 'int', 'kind': 'parameter'},
   'b_float': {'type': 'float', 'kind': 'parameter'},
   'c_double': {'type': 'double', 'kind': 'parameter'},
   'char_pointer': {'type': '*char', 'kind': 'parameter'},
   'e_int': {'type': 'int', 'kind': 'variable'},
   'f_float': {'type': 'float', 'kind': 'variable'},
   'g_double': {'type'

In [6]:
#!/usr/bin/env python3

import sys,os
from elftools.elf.elffile import ELFFile
from elftools.elf.segments import Segment



filePath = './../../binaries/c_many/stacktest'


In [7]:
from capstone import *

address_inst = {}
with open(filePath, 'rb') as f:
    elf = ELFFile(f)
    code = elf.get_section_by_name('.text')
    ops = code.data()
    print('code data size: ',code.data_size)
    addr = code['sh_addr']
    md = Cs(CS_ARCH_X86, CS_MODE_64)
    for i in md.disasm(ops, addr):        
        address_inst[hex(i.address)] = i

code data size:  1328


In [8]:
from collections import defaultdict
import posixpath


In [9]:
addr_lineProgram ={}
addr_sourceFile = {}
with open(filePath, 'rb') as f:
    elffile = ELFFile(f)

    if not elffile.has_dwarf_info():
        print('  file has no DWARF info')
        exit(0)

    dwarfinfo = elffile.get_dwarf_info()
    for CU in dwarfinfo.iter_CUs():
        print('  Found a compile unit at offset %s, length %s' % (
            CU.cu_offset, CU['unit_length']))

        # Every compilation unit in the DWARF information may or may not
        # have a corresponding line program in .debug_line.
        line_program = dwarfinfo.line_program_for_CU(CU)
        if line_program is None:
            print('  DWARF info is missing a line program for this CU')
            continue

        # Print a reverse mapping of filename -> #entries
        filename_map = line_entry_mapping(line_program)
        for line_entry in line_program.get_entries():

            if line_entry.state!=None:
                addr_lineProgram[hex(line_entry.state.address)] = line_entry
                addr_sourceFile [hex(line_entry.state.address)] = filename_map

        
        


  Found a compile unit at offset 0, length 890
  Found a compile unit at offset 894, length 517
  Found a compile unit at offset 1415, length 446


In [10]:
# print(addr_lineProgram)
# address_inst

def getSource(sourceFileName, row , col):
    basePath = "/home/nahid/reverse/binaries/c_many/"
    sourceFilePath = os.path.join(basePath , sourceFileName)
    sourceFile = open(sourceFilePath, "r")
    fileContent = sourceFile.readlines()
    
    row_content =  fileContent[row-1]
    row_content = row_content[:(col-1)] + '|'+row_content[(col-1)]+'|' +row_content[col:]
    
    return row_content


In [11]:

# FUNCTION_DECL
# https://stackoverflow.com/questions/43460605/function-boundary-identification-using-libclang
# https://eli.thegreenplace.net/2011/07/03/parsing-c-in-python-with-clang


import clang.cindex



def get_all_var_types(source_path):
    srcFileName = source_path.split('/')[-1]

    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.VAR_DECL:
            # print(dir(f))
            print(f.extent.start.file.name)
            originFileName = f.extent.start.file.name.split('/')[-1]
            print(originFileName)
            print(f.displayname)
            print(f.type.spelling)
            print(f.extent.start.line,f.extent.start.column)
            print('\n')
            
            

   

def get_all_function_types(source_path):
    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.FUNCTION_DECL:
            # print(dir(f))
            print(f.displayname)
            print('function name: ',( f.spelling))
            print('Returns: ',(f.result_type.spelling))
            
            
            arg_len = len(list(f.type.argument_types()))
            if arg_len>0:
                arg_types = list(f.type.argument_types())
                for arg_type in arg_types:
                    print('arg_type:',arg_type.spelling)
                args = list(f.get_arguments())
                for arg in args:
                    print('arg:',arg.spelling)


            print("\n\n\n")

            
def get_function_boundaries(source_path):
    
    function_boundary_by_name = {}
    idx = clang.cindex.Index.create()
    tu = idx.parse(source_path)
    
    for f in tu.cursor.walk_preorder():
        if f.kind == clang.cindex.CursorKind.FUNCTION_DECL:

            function_name = f.displayname.split('(')[0]
            function_boundary_by_name[function_name]={}
            function_boundary_by_name[function_name] = { 'src_path':f.extent.start.file.name,
                              'src_file':f.extent.start.file.name.split('/')[-1],
                              'start_line':f.extent.start.line,
                              'start_col':f.extent.start.column,
                              'end_line':f.extent.end.line,
                              'end_col':f.extent.end.column}
    return function_boundary_by_name

def get_containing_function(source_file_path, line, col=0):
    function_boundary_by_name = get_function_boundaries(source_file_path)
    
    for function_name, item in function_boundary_by_name.items():
        if item['src_path'] == source_file_path:
            if line>= item['start_line'] and line<= item['end_line']:
                return function_name
        

# _=get_all_var_types('/home/nahid/reverse/binaries/c_many/stack.c' )
# print(_)

_=get_all_function_types('/home/nahid/reverse/binaries/c_many/stack.c' )
print(_)

remove(const char *)
function name:  remove
Returns:  int
arg_type: const char *
arg: __filename




rename(const char *, const char *)
function name:  rename
Returns:  int
arg_type: const char *
arg_type: const char *
arg: __old
arg: __new




renameat(int, const char *, int, const char *)
function name:  renameat
Returns:  int
arg_type: int
arg_type: const char *
arg_type: int
arg_type: const char *
arg: __oldfd
arg: __old
arg: __newfd
arg: __new




fclose(FILE *)
function name:  fclose
Returns:  int
arg_type: FILE *
arg: __stream




tmpfile()
function name:  tmpfile
Returns:  FILE *




tmpnam(char *)
function name:  tmpnam
Returns:  char *
arg_type: char[20]
arg: 




tmpnam_r(char *)
function name:  tmpnam_r
Returns:  char *
arg_type: char[20]
arg: __s




tempnam(const char *, const char *)
function name:  tempnam
Returns:  char *
arg_type: const char *
arg_type: const char *
arg: __dir
arg: __pfx




fflush(FILE *)
function name:  fflush
Returns:  int
arg_type: FILE *
arg: __s

arg_type: const char *
arg_type: char **
arg_type: int
arg: __nptr
arg: __endptr
arg: __base




strtoq(const char *restrict, char **restrict, int)
function name:  strtoq
Returns:  long long
arg_type: const char *restrict
arg_type: char **restrict
arg_type: int
arg: __nptr
arg: __endptr
arg: __base




strtouq(const char *restrict, char **restrict, int)
function name:  strtouq
Returns:  unsigned long long
arg_type: const char *restrict
arg_type: char **restrict
arg_type: int
arg: __nptr
arg: __endptr
arg: __base




strtoll(const char *restrict, char **restrict, int)
function name:  strtoll
Returns:  long long
arg_type: const char *
arg_type: char **
arg_type: int
arg: __nptr
arg: __endptr
arg: __base




strtoull(const char *restrict, char **restrict, int)
function name:  strtoull
Returns:  unsigned long long
arg_type: const char *
arg_type: char **
arg_type: int
arg: __nptr
arg: __endptr
arg: __base




l64a(long)
function name:  l64a
Returns:  char *
arg_type: long
arg: __n




a64l





abs(int)
function name:  abs
Returns:  int
arg_type: int
arg: __x




labs(long)
function name:  labs
Returns:  long
arg_type: long
arg: __x




llabs(long long)
function name:  llabs
Returns:  long long
arg_type: long long
arg: __x




div(int, int)
function name:  div
Returns:  div_t
arg_type: int
arg_type: int
arg: __numer
arg: __denom




ldiv(long, long)
function name:  ldiv
Returns:  ldiv_t
arg_type: long
arg_type: long
arg: __numer
arg: __denom




lldiv(long long, long long)
function name:  lldiv
Returns:  lldiv_t
arg_type: long long
arg_type: long long
arg: __numer
arg: __denom




ecvt(double, int, int *restrict, int *restrict)
function name:  ecvt
Returns:  char *
arg_type: double
arg_type: int
arg_type: int *restrict
arg_type: int *restrict
arg: __value
arg: __ndigit
arg: __decpt
arg: __sign




fcvt(double, int, int *restrict, int *restrict)
function name:  fcvt
Returns:  char *
arg_type: double
arg_type: int
arg_type: int *restrict
arg_type: int *restrict
arg: __value

In [12]:

dir_path = './../../binaries/c_many/'

with open('stacktest.s', 'w') as outFile:
    # outFile.write('file contents\n')
    lastSource = ""
    for address in address_inst:
        inst = address_inst[address]
        instrctionCode = (address+":\t"+ inst.mnemonic+" "+inst.op_str).ljust(45)
        
        if address in addr_lineProgram:
            line = addr_lineProgram[address]

            srcFileName =list(addr_sourceFile[address].keys())[0] #TODO not single file always 
            
            if srcFileName!=lastSource:
                outFile.write("\n"+ '#'*100+"\n"+ srcFileName.rjust(45) +'\n'+'#'*100+ "\n\n")
                lastSource = srcFileName
            
            sourceCode = getSource(srcFileName,addr_lineProgram[address].state.line, addr_lineProgram[address].state.column)
            function_name = get_containing_function(dir_path+srcFileName ,addr_lineProgram[address].state.line , addr_lineProgram[address].state.column)
            print(function_name)
            
            if '\n' not in  sourceCode:
                sourceCode+=sourceCode+"\n"
            outFile.write(instrctionCode+"#"+ sourceCode  )
            print(instrctionCode+"#"+ sourceCode)
            
        else:
            
            outFile.write(instrctionCode+ '\n'  )
            print(instrctionCode)
    

0x10c0:	endbr64                              
0x10c4:	xor ebp, ebp                         
0x10c6:	mov r9, rdx                          
0x10c9:	pop rsi                              
0x10ca:	mov rdx, rsp                         
0x10cd:	and rsp, 0xfffffffffffffff0          
0x10d1:	push rax                             
0x10d2:	push rsp                             
0x10d3:	xor r8d, r8d                         
0x10d6:	xor ecx, ecx                         
0x10d8:	lea rdi, [rip + 0x2af]               
0x10df:	call qword ptr [rip + 0x2ef3]        
0x10e5:	hlt                                  
0x10e6:	nop word ptr cs:[rax + rax]          
0x10f0:	lea rdi, [rip + 0x2f19]              
0x10f7:	lea rax, [rip + 0x2f12]              
0x10fe:	cmp rax, rdi                         
0x1101:	je 0x1118                            
0x1103:	mov rax, qword ptr [rip + 0x2ed6]    
0x110a:	test rax, rax                        
0x110d:	je 0x1118                            
0x110f:	jmp rax                   

pop
0x1347:	mov eax, dword ptr [rbp - 0x14]      #    return |n|umber;

pop
0x134a:	leave                                #|}|

0x134b:	ret                                  
fake
0x134c:	endbr64                              #int fake(int a , int b)|{|

0x1350:	push rbp                             
0x1351:	mov rbp, rsp                         
0x1354:	mov dword ptr [rbp - 0x14], edi      
0x1357:	mov dword ptr [rbp - 0x18], esi      
fake
0x135a:	mov dword ptr [rbp - 0x10], 0xa      #    int |d| =10;

fake
0x1361:	mov dword ptr [rbp - 0xc], 0xb       #    int |e| =11;

fake
0x1368:	mov dword ptr [rbp - 8], 0xc         #    int |f| =12;

fake
0x136f:	mov edx, dword ptr [rbp - 0x14]      #    int g = a|+|b+d+e+f;

0x1372:	mov eax, dword ptr [rbp - 0x18]      
0x1375:	add edx, eax                         
fake
0x1377:	mov eax, dword ptr [rbp - 0x10]      #    int g = a+b|+|d+e+f;

0x137a:	add edx, eax                         
fake
0x137c:	mov eax, dword ptr [rbp - 0xc]       #    int g = a+

add
0x159c:	pop rbp                              #|}|

0x159d:	ret                                  
addf
0x159e:	endbr64                              #float addf(float a, float b)|{|

0x15a2:	push rbp                             
0x15a3:	mov rbp, rsp                         
0x15a6:	movss dword ptr [rbp - 4], xmm0      
0x15ab:	movss dword ptr [rbp - 8], xmm1      
addf
0x15b0:	movss xmm0, dword ptr [rbp - 4]      #    return a|+|b;

0x15b5:	addss xmm0, dword ptr [rbp - 8]      
addf
0x15ba:	pop rbp                              #|}|

0x15bb:	ret                                  
substract
0x15bc:	endbr64                              #int substract(int a, int b)|{|

0x15c0:	push rbp                             
0x15c1:	mov rbp, rsp                         
0x15c4:	mov dword ptr [rbp - 4], edi         
0x15c7:	mov dword ptr [rbp - 8], esi         
substract
0x15ca:	mov eax, dword ptr [rbp - 4]         #    return a|-|b;

0x15cd:	sub eax, dword ptr [rbp - 8]         
substract
0x15d0:	po